# Builder Pattern Tutorial 🏗️

## What is the Builder Pattern?

The Builder Pattern lets you construct complex objects step by step. It allows you to produce different types and representations of an object using the same construction code. Think of it as following a recipe - you add ingredients one by one to create the final dish.

**Real-world analogy**: Building a house. You don't construct a house all at once - you lay the foundation, build the walls, add the roof, install plumbing, etc. Each step is separate, and you can customize each part (brick vs. wood walls, tile vs. metal roof).

## Table of Contents
1. [What is the Builder Pattern?](#what-is-the-builder-pattern)
2. [Why Do We Need It?](#why-do-we-need-it)
3. [Simple Implementation](#simple-implementation)
4. [Understanding the Implementation](#understanding-the-implementation)
5. [SQL Query Builder (Advanced)](#advanced-example-sql-query-builder)
6. [When to Use (and When NOT to Use)](#when-to-use-and-when-not-to-use)

## Why Do We Need It?

Let's see what happens when we try to create complex objects without the builder pattern:

In [ ]:
# Problem: Complex constructor with many parameters

class Computer:
    def __init__(self, cpu, memory, storage, graphics_card=None, 
                 sound_card=None, network_card=None, bluetooth=False, 
                 wifi=False, case_color="black", operating_system=None,
                 monitor=None, keyboard=None, mouse=None):
        self.cpu = cpu
        self.memory = memory
        self.storage = storage
        self.graphics_card = graphics_card
        self.sound_card = sound_card
        self.network_card = network_card
        self.bluetooth = bluetooth
        self.wifi = wifi
        self.case_color = case_color
        self.operating_system = operating_system
        self.monitor = monitor
        self.keyboard = keyboard
        self.mouse = mouse
    
    def __str__(self):
        return f"Computer with {self.cpu}, {self.memory}, {self.storage}"

# Problems with this approach:

# 1. Long, confusing constructor calls
basic_computer = Computer(
    "Intel i5", "8GB", "256GB SSD", None, None, None, 
    False, True, "black", "Windows 11", None, None, None
)

# 2. Easy to mix up parameter order
gaming_computer = Computer(
    "Intel i9", "32GB", "1TB SSD", "RTX 4080", 
    "Creative Sound Blaster", "Gigabit Ethernet", 
    True, True, "RGB", "Windows 11", 
    "27-inch 4K", "Mechanical", "Gaming Mouse"
)

# 3. Need multiple constructors for different configurations
class ComputerWithMultipleConstructors:
    def __init__(self, cpu, memory, storage):
        self.cpu = cpu
        self.memory = memory
        self.storage = storage
        # Many more attributes...
    
    @classmethod
    def create_gaming_computer(cls, cpu, memory, storage, graphics_card):
        computer = cls(cpu, memory, storage)
        computer.graphics_card = graphics_card
        computer.sound_card = "High-end"
        # More gaming-specific setup...
        return computer
    
    @classmethod
    def create_office_computer(cls, cpu, memory, storage):
        computer = cls(cpu, memory, storage)
        computer.graphics_card = "Integrated"
        # More office-specific setup...
        return computer

# This leads to an explosion of factory methods!

print(f"Basic: {basic_computer}")
print(f"Gaming: {gaming_computer}")

## Simple Implementation

Let's solve this with the Builder Pattern:

In [ ]:
from abc import ABC, abstractmethod

# Product
class Computer:
    def __init__(self):
        self.cpu = None
        self.memory = None
        self.storage = None
        self.graphics_card = None
        self.sound_card = None
        self.network_card = None
        self.bluetooth = False
        self.wifi = False
        self.case_color = "black"
        self.operating_system = None
        self.monitor = None
        self.keyboard = None
        self.mouse = None
        self.components = []
    
    def add_component(self, component):
        self.components.append(component)
    
    def get_specs(self):
        specs = []
        specs.append(f"CPU: {self.cpu}")
        specs.append(f"Memory: {self.memory}")
        specs.append(f"Storage: {self.storage}")
        
        if self.graphics_card:
            specs.append(f"Graphics: {self.graphics_card}")
        if self.sound_card:
            specs.append(f"Sound: {self.sound_card}")
        if self.network_card:
            specs.append(f"Network: {self.network_card}")
        if self.bluetooth:
            specs.append("Bluetooth: Yes")
        if self.wifi:
            specs.append("WiFi: Yes")
        if self.operating_system:
            specs.append(f"OS: {self.operating_system}")
        
        return "\n".join(specs)

# Abstract Builder
class ComputerBuilder(ABC):
    def __init__(self):
        self.computer = Computer()
    
    @abstractmethod
    def build_cpu(self):
        pass
    
    @abstractmethod
    def build_memory(self):
        pass
    
    @abstractmethod
    def build_storage(self):
        pass
    
    def build_graphics_card(self):
        pass  # Optional component
    
    def build_sound_card(self):
        pass  # Optional component
    
    def build_network_features(self):
        pass  # Optional component
    
    def install_operating_system(self):
        pass  # Optional component
    
    def get_computer(self):
        return self.computer

# Concrete Builders
class GamingComputerBuilder(ComputerBuilder):
    def build_cpu(self):
        self.computer.cpu = "Intel Core i9-13900K"
        return self
    
    def build_memory(self):
        self.computer.memory = "32GB DDR5"
        return self
    
    def build_storage(self):
        self.computer.storage = "1TB NVMe SSD + 2TB HDD"
        return self
    
    def build_graphics_card(self):
        self.computer.graphics_card = "NVIDIA RTX 4080"
        return self
    
    def build_sound_card(self):
        self.computer.sound_card = "Creative Sound BlasterX AE-5"
        return self
    
    def build_network_features(self):
        self.computer.network_card = "Gigabit Ethernet"
        self.computer.wifi = True
        self.computer.bluetooth = True
        return self
    
    def install_operating_system(self):
        self.computer.operating_system = "Windows 11 Pro"
        return self

class OfficeComputerBuilder(ComputerBuilder):
    def build_cpu(self):
        self.computer.cpu = "Intel Core i5-13400"
        return self
    
    def build_memory(self):
        self.computer.memory = "16GB DDR4"
        return self
    
    def build_storage(self):
        self.computer.storage = "512GB SSD"
        return self
    
    def build_graphics_card(self):
        self.computer.graphics_card = "Integrated Intel UHD"
        return self
    
    def build_network_features(self):
        self.computer.network_card = "Gigabit Ethernet"
        self.computer.wifi = True
        self.computer.bluetooth = False
        return self
    
    def install_operating_system(self):
        self.computer.operating_system = "Windows 11 Business"
        return self

In [ ]:
# Director (optional - can be done directly with builder)
class ComputerDirector:
    def __init__(self, builder: ComputerBuilder):
        self.builder = builder
    
    def build_basic_computer(self):
        return (self.builder
                .build_cpu()
                .build_memory()
                .build_storage()
                .install_operating_system()
                .get_computer())
    
    def build_complete_computer(self):
        return (self.builder
                .build_cpu()
                .build_memory()
                .build_storage()
                .build_graphics_card()
                .build_sound_card()
                .build_network_features()
                .install_operating_system()
                .get_computer())

# Using the Builder Pattern
print("--- Building Computers with Builder Pattern ---")

# Building a gaming computer with director
gaming_builder = GamingComputerBuilder()
director = ComputerDirector(gaming_builder)
gaming_computer = director.build_complete_computer()

print("Gaming Computer:")
print(gaming_computer.get_specs())
print()

# Building an office computer directly with builder (fluent interface)
office_computer = (OfficeComputerBuilder()
                  .build_cpu()
                  .build_memory()
                  .build_storage()
                  .build_graphics_card()
                  .build_network_features()
                  .install_operating_system()
                  .get_computer())

print("Office Computer:")
print(office_computer.get_specs())
print()

# Building a custom computer (cherry-picking features)
custom_computer = (GamingComputerBuilder()
                  .build_cpu()
                  .build_memory()
                  .build_storage()
                  # Skip graphics card for now
                  .build_network_features()
                  .install_operating_system()
                  .get_computer())

print("Custom Computer (no graphics card):")
print(custom_computer.get_specs())

## Understanding the Implementation

### Key Concepts:

1. **Product**: The complex object being built (Computer)
2. **Builder**: Abstract interface defining steps to build the product
3. **Concrete Builder**: Implements the Builder interface for specific product types
4. **Director**: Optional class that defines the order of building steps
5. **Fluent Interface**: Method chaining for readable code

### Benefits:
- **Readable Code**: Clear step-by-step construction
- **Flexible**: Can create different representations of the same product
- **Immutable Objects**: Can build immutable objects step by step
- **Reusable**: Directors can be reused with different builders

## Advanced Example: SQL Query Builder

Let's create a practical example - a SQL query builder:

In [ ]:
from typing import List, Optional

class SQLQuery:
    def __init__(self):
        self.query_type = None
        self.table = None
        self.columns = []
        self.values = []
        self.where_conditions = []
        self.joins = []
        self.order_by = []
        self.group_by = []
        self.having = []
        self.limit_value = None
        self.offset_value = None
    
    def to_sql(self) -> str:
        if self.query_type == "SELECT":
            return self._build_select_query()
        elif self.query_type == "INSERT":
            return self._build_insert_query()
        elif self.query_type == "UPDATE":
            return self._build_update_query()
        elif self.query_type == "DELETE":
            return self._build_delete_query()
        else:
            raise ValueError("Unknown query type")
    
    def _build_select_query(self) -> str:
        parts = []
        
        # SELECT clause
        columns = ", ".join(self.columns) if self.columns else "*"
        parts.append(f"SELECT {columns}")
        
        # FROM clause
        parts.append(f"FROM {self.table}")
        
        # JOIN clauses
        for join in self.joins:
            parts.append(join)
        
        # WHERE clause
        if self.where_conditions:
            parts.append(f"WHERE {' AND '.join(self.where_conditions)}")
        
        # GROUP BY clause
        if self.group_by:
            parts.append(f"GROUP BY {', '.join(self.group_by)}")
        
        # HAVING clause
        if self.having:
            parts.append(f"HAVING {' AND '.join(self.having)}")
        
        # ORDER BY clause
        if self.order_by:
            parts.append(f"ORDER BY {', '.join(self.order_by)}")
        
        # LIMIT clause
        if self.limit_value:
            parts.append(f"LIMIT {self.limit_value}")
        
        # OFFSET clause
        if self.offset_value:
            parts.append(f"OFFSET {self.offset_value}")
        
        return " ".join(parts)
    
    def _build_insert_query(self) -> str:
        columns = ", ".join(self.columns)
        values = ", ".join([f"'{v}'" if isinstance(v, str) else str(v) for v in self.values])
        return f"INSERT INTO {self.table} ({columns}) VALUES ({values})"
    
    def _build_update_query(self) -> str:
        set_clauses = []
        for col, val in zip(self.columns, self.values):
            val_str = f"'{val}'" if isinstance(val, str) else str(val)
            set_clauses.append(f"{col} = {val_str}")
        
        parts = [f"UPDATE {self.table}", f"SET {', '.join(set_clauses)}"]
        
        if self.where_conditions:
            parts.append(f"WHERE {' AND '.join(self.where_conditions)}")
        
        return " ".join(parts)
    
    def _build_delete_query(self) -> str:
        parts = [f"DELETE FROM {self.table}"]
        
        if self.where_conditions:
            parts.append(f"WHERE {' AND '.join(self.where_conditions)}")
        
        return " ".join(parts)

class QueryBuilder:
    def __init__(self):
        self.query = SQLQuery()
    
    def select(self, *columns) -> 'QueryBuilder':
        self.query.query_type = "SELECT"
        self.query.columns = list(columns)
        return self
    
    def insert_into(self, table: str) -> 'QueryBuilder':
        self.query.query_type = "INSERT"
        self.query.table = table
        return self
    
    def update(self, table: str) -> 'QueryBuilder':
        self.query.query_type = "UPDATE"
        self.query.table = table
        return self
    
    def delete_from(self, table: str) -> 'QueryBuilder':
        self.query.query_type = "DELETE"
        self.query.table = table
        return self
    
    def from_table(self, table: str) -> 'QueryBuilder':
        self.query.table = table
        return self
    
    def values(self, **kwargs) -> 'QueryBuilder':
        self.query.columns = list(kwargs.keys())
        self.query.values = list(kwargs.values())
        return self
    
    def set(self, **kwargs) -> 'QueryBuilder':
        self.query.columns = list(kwargs.keys())
        self.query.values = list(kwargs.values())
        return self
    
    def where(self, condition: str) -> 'QueryBuilder':
        self.query.where_conditions.append(condition)
        return self
    
    def join(self, table: str, on: str, join_type: str = "INNER") -> 'QueryBuilder':
        self.query.joins.append(f"{join_type} JOIN {table} ON {on}")
        return self
    
    def left_join(self, table: str, on: str) -> 'QueryBuilder':
        return self.join(table, on, "LEFT")
    
    def right_join(self, table: str, on: str) -> 'QueryBuilder':
        return self.join(table, on, "RIGHT")
    
    def order_by(self, *columns) -> 'QueryBuilder':
        self.query.order_by.extend(columns)
        return self
    
    def group_by(self, *columns) -> 'QueryBuilder':
        self.query.group_by.extend(columns)
        return self
    
    def having(self, condition: str) -> 'QueryBuilder':
        self.query.having.append(condition)
        return self
    
    def limit(self, count: int) -> 'QueryBuilder':
        self.query.limit_value = count
        return self
    
    def offset(self, count: int) -> 'QueryBuilder':
        self.query.offset_value = count
        return self
    
    def build(self) -> str:
        return self.query.to_sql()

# Testing the SQL Query Builder
print("--- SQL Query Builder Examples ---")

# Complex SELECT query
select_query = (QueryBuilder()
               .select("u.name", "u.email", "p.title", "COUNT(c.id) as comment_count")
               .from_table("users u")
               .left_join("posts p", "u.id = p.user_id")
               .left_join("comments c", "p.id = c.post_id")
               .where("u.active = 1")
               .where("p.published = 1")
               .group_by("u.id", "p.id")
               .having("comment_count > 5")
               .order_by("comment_count DESC", "u.name ASC")
               .limit(10)
               .build())

print("Complex SELECT:")
print(select_query)
print()

# INSERT query
insert_query = (QueryBuilder()
               .insert_into("users")
               .values(name="John Doe", email="john@example.com", age=30)
               .build())

print("INSERT:")
print(insert_query)
print()

# UPDATE query
update_query = (QueryBuilder()
               .update("users")
               .set(email="newemail@example.com", age=31)
               .where("id = 1")
               .build())

print("UPDATE:")
print(update_query)
print()

# DELETE query
delete_query = (QueryBuilder()
               .delete_from("users")
               .where("active = 0")
               .where("last_login < '2023-01-01'")
               .build())

print("DELETE:")
print(delete_query)

## Builder Pattern with Validation

Let's create a builder that validates the object as it's being built:

In [ ]:
from dataclasses import dataclass
from typing import Optional
import re

@dataclass
class User:
    username: str
    email: str
    age: int
    first_name: Optional[str] = None
    last_name: Optional[str] = None
    phone: Optional[str] = None
    is_active: bool = True
    
    def __str__(self):
        name = f"{self.first_name} {self.last_name}".strip() or "N/A"
        return f"User(username={self.username}, name={name}, email={self.email})"

class UserBuilder:
    def __init__(self):
        self._username = None
        self._email = None
        self._age = None
        self._first_name = None
        self._last_name = None
        self._phone = None
        self._is_active = True
    
    def username(self, username: str) -> 'UserBuilder':
        if not username or len(username) < 3:
            raise ValueError("Username must be at least 3 characters long")
        if not re.match(r'^[a-zA-Z0-9_]+$', username):
            raise ValueError("Username can only contain letters, numbers, and underscores")
        self._username = username
        return self
    
    def email(self, email: str) -> 'UserBuilder':
        email_pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
        if not re.match(email_pattern, email):
            raise ValueError("Invalid email format")
        self._email = email
        return self
    
    def age(self, age: int) -> 'UserBuilder':
        if not isinstance(age, int) or age < 0 or age > 150:
            raise ValueError("Age must be a valid integer between 0 and 150")
        self._age = age
        return self
    
    def first_name(self, first_name: str) -> 'UserBuilder':
        if first_name and len(first_name.strip()) == 0:
            raise ValueError("First name cannot be empty")
        self._first_name = first_name.strip() if first_name else None
        return self
    
    def last_name(self, last_name: str) -> 'UserBuilder':
        if last_name and len(last_name.strip()) == 0:
            raise ValueError("Last name cannot be empty")
        self._last_name = last_name.strip() if last_name else None
        return self
    
    def phone(self, phone: str) -> 'UserBuilder':
        if phone:
            # Simple phone validation (adjust as needed)
            phone_clean = re.sub(r'[^\d+]', '', phone)
            if len(phone_clean) < 10:
                raise ValueError("Phone number must have at least 10 digits")
        self._phone = phone
        return self
    
    def active(self, is_active: bool = True) -> 'UserBuilder':
        self._is_active = is_active
        return self
    
    def inactive(self) -> 'UserBuilder':
        return self.active(False)
    
    def build(self) -> User:
        # Final validation
        if not self._username:
            raise ValueError("Username is required")
        if not self._email:
            raise ValueError("Email is required")
        if self._age is None:
            raise ValueError("Age is required")
        
        return User(
            username=self._username,
            email=self._email,
            age=self._age,
            first_name=self._first_name,
            last_name=self._last_name,
            phone=self._phone,
            is_active=self._is_active
        )
    
    def reset(self) -> 'UserBuilder':
        """Reset the builder to create a new user"""
        self.__init__()
        return self

# Testing the validated builder
print("--- Testing Validated User Builder ---")

# Valid user
try:
    user1 = (UserBuilder()
             .username("john_doe")
             .email("john@example.com")
             .age(25)
             .first_name("John")
             .last_name("Doe")
             .phone("+1-555-123-4567")
             .active()
             .build())
    print(f"Created: {user1}")
except ValueError as e:
    print(f"Error: {e}")

# Minimal valid user
try:
    user2 = (UserBuilder()
             .username("alice_wonderland")
             .email("alice@wonderland.com")
             .age(30)
             .build())
    print(f"Created: {user2}")
except ValueError as e:
    print(f"Error: {e}")

# Invalid user (bad email)
try:
    user3 = (UserBuilder()
             .username("baduser")
             .email("not-an-email")
             .age(25)
             .build())
    print(f"Created: {user3}")
except ValueError as e:
    print(f"Error: {e}")

# Invalid user (missing required field)
try:
    user4 = (UserBuilder()
             .username("incomplete")
             .age(25)
             # Missing email
             .build())
    print(f"Created: {user4}")
except ValueError as e:
    print(f"Error: {e}")

## When to Use (and When NOT to Use)

### Use Builder Pattern when:
- You need to create objects with many optional parameters
- The construction process must allow different representations for the object
- You want to isolate complex construction logic from the product itself
- You need to construct objects step by step or with validation at each step
- You want to create immutable objects that need complex initialization
- The order of construction steps matters

### Don't use when:
- The object has few properties or simple construction
- The construction process doesn't vary much
- Simple factory methods or constructors are sufficient
- The overhead of creating builder classes isn't justified

### Real-world applications:
- SQL query builders (Hibernate, SQLAlchemy)
- HTTP request builders (OkHttp, Retrofit)
- Configuration objects (Spring Boot, Apache Commons Configuration)
- GUI component builders (Swing, JavaFX)
- Document builders (PDF, Word, HTML generators)
- Test data builders (for unit testing)
- API request/response builders
- Game object creation (characters, levels, items)